# German Wikipedia Power Analysis

Using the registration, activation, and retention statistics from 2020–21 data gathering, do a simplified power analysis for German Wikipedia. We'll be using the `pwr` package in R for these estimates.

Note that our statistics include three months of some proportion of users getting some variation of the Growth Features, as that Wikipedia has had the features since August/September 2021. Since we're using 12-month averages, I think we're making reasonable estimates.

In [5]:
library(tidyverse)
library(pwr)
library(data.table)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()



In [2]:
## Lists of effect sizes that we're estimating for.
## [2%, 30%] changes in activation
## [-30%, 30%] changes in retention

activation_deltas <- c(1.02, 1.05, 1.1, 1.2, 1.3)
retention_deltas <- c(0.7, 0.8, 0.9, 0.95, 0.98, 1.0, 1.02, 1.05, 1.1, 1.2, 1.3)

In [6]:
calc_edit_power <- function(base_proportion, proportion_deltas, N_control,
                           sig.level = 0.05, power = 0.8) {
    ## Given a base proportion and N users in a control group, for each delta
    ## (change in proportion) calculate the number of participants in the treatment
    ## group that's needed to achieve a significance level of `sig.level` and power of `power`
    
    ## Treatment group number of usres
    treat_counts <- c()
    
    for(prop_delta in proportion_deltas) {
        treat_counts <- append(
            treat_counts,
            tryCatch(
                pwr.2p2n.test(
                    h = ES.h(
                        p1 = base_proportion,
                        p2 = base_proportion * prop_delta
                    ),
                    n1 = N_control,
                    sig.level = sig.level,
                    power = power,
                    alternative = 'two.sided'
                )$n2,
                error = function(e) { return(NA) }
            )
        )
      }
      tibble(
          delta = proportion_deltas,
          n_treatment = treat_counts
      )
    }

In [20]:
calc_power_5050 = function(edit_base, edit_deltas,
                           sig.level = 0.05, power = 0.8) {
  ## Given an average proportion of registrations
  ## who make an edit during a given time span (`edit_base`), and a set of changes in said
  ## proportion (`edit_deltas`), calculate the number of participants needed in order to
  ## run an experiment with a significance level of `sig.level` and power of `power`.
  
  deltas = edit_deltas;
  ns = c();
  
  for(edit_delta in edit_deltas) {
    ns = append(ns,
                tryCatch(pwr.p.test(h = ES.h(p1 = edit_base,
                                             p2 = edit_base*edit_delta),
                                    sig.level = sig.level,
                                    power = power,
                                    alternative = 'two.sided')$n,
                         error = function(e) return(NA)));
  }
  data.table(delta = deltas, n = ns);
}

In [17]:
## Configuration variables from German Wikipedia

# monthly registration rate
registration_rate = 8057

# activation rate on mobile, the lower of the two (desktop is 43.2%)
activation_rate = 0.41

# retention rate on mobile, the lower of the two (desktop is 18.8%)
retention_rate = 0.107

# we're assuming 20% assignment to the Control group
control_group_registrations = registration_rate * 0.2
treatment_group_registrations = registration_rate * 0.8

control_group_activations = control_group_registrations * activation_rate
treatment_group_activations = treatment_group_registrations * activation_rate

In [12]:
activation_counts = calc_edit_power(activation_rate, activation_deltas, control_group_registrations)

In [16]:
activation_counts %>%
    mutate(n_months = n_treatment / treatment_group_registrations,
           n_days = ceiling(n_months * 30))

delta,n_treatment,n_months,n_days
<dbl>,<dbl>,<dbl>,<dbl>
1.02,NA,NA,NA
1.05,NA,NA,NA
1.10,3944.5589,0.61197699,19
1.20,351.1877,0.05448488,2
1.30,139.7709,0.02168470,1


In [18]:
retention_counts = calc_edit_power(retention_rate, retention_deltas, control_group_activations)

In [19]:
retention_counts %>%
    mutate(n_months = n_treatment / treatment_group_activations,
           n_days = ceiling(n_months * 30))

delta,n_treatment,n_months,n_days
<dbl>,<dbl>,<dbl>,<dbl>
0.70,11613.99,4.394751,132
0.80,NA,NA,NA
0.90,NA,NA,NA
0.95,NA,NA,NA
0.98,NA,NA,NA
1.00,NA,NA,NA
1.02,NA,NA,NA
1.05,NA,NA,NA
1.10,NA,NA,NA


In [21]:
activation_counts_5050 = calc_power_5050(activation_rate, activation_deltas)

In [23]:
activation_counts_5050 %>%
    mutate(n_months = n / registration_rate,
           n_days = ceiling(n_months * 30))

delta,n,n_months,n_days
<dbl>,<dbl>,<dbl>,<dbl>
1.02,28320.2243,3.51498378,106
1.05,4549.6715,0.56468555,17
1.10,1144.0441,0.14199381,5
1.20,288.3458,0.03578823,2
1.30,128.6150,0.01596313,1


In [24]:
retention_counts_5050 = calc_power_5050(retention_rate, retention_deltas)

In [25]:
retention_counts_5050 %>%
    mutate(n_months = n / (registration_rate * activation_rate),
           n_days = ceiling(n_months * 30))

delta,n,n_months,n_days
<dbl>,<dbl>,<dbl>,<dbl>
0.70,625.1137,0.1892352,6
0.80,1487.1790,0.4502006,14
0.90,6256.1880,1.8938805,57
0.95,25619.5246,7.7555722,233
0.98,162315.2034,49.1362467,1475
1.00,NA,NA,NA
1.02,165198.0996,50.0089604,1501
1.05,26772.8731,8.1047152,244
1.10,6833.2030,2.0685551,63
